In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from itertools import combinations
from scipy.stats import ttest_ind, kendalltau
from sklearn.feature_selection import SelectKBest, f_classif
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

df = pd.read_csv('stud_math.csv')
df = df.drop('studytime, granular', axis=1)
%matplotlib inline

## Часть третья.  Числовая

In [ ]:
# Разделим колонки по типам переменных, содержащихся в них.

cat_cols = df.columns[df.dtypes == 'O'].to_list()
num_cols_y = df.columns[df.dtypes == 'float64'].to_list()

### Score
Сначала посмотрим на нашу целевую переменную **score**

In [ ]:
df.score.hist()

Видим две части: на правой распределение очень похожее на нормальное, левая содержит значение 0. При стобальной оценке на экзамене, оценка 0 скорее значит, что ученик не писал работу совсем, либо оценка не попала в данные.

In [ ]:
df[df.score==0].head()

*Здесь вывод ограничен пятью строками для удобства чтения, но на всех  
остальных данных поле **'absences'** также содержит нулевое значение (и один NaN)  
при нулевом  **'score'**. Источник данных о  пропущенных занятиях и итоговая  
оценка это школа, остальные данные могли быть получене из других источников,  
здесь явно какая-то аномалия. Причем нулевая оценка по экзамену никак не  
коррелирует с другими данными, например вряд ли выглядит правдоподобным желание  
получить высшее образование и нулевая оценка по математике. Мне кажется более  
правильным удалить строки с нулевым значением **'score'**, чем заменять эти  
значения, например на среднее или обучать модель на нулевом значении. По тем  
же соображениям удалим строки (6) с пропусками в целевой переменной.*

In [ ]:
df = df[df.score != 0]
df = df.dropna(subset=['score'])

### Absences
Колонка с пропусками занятий absences содержит два выброса - 385 и 212. Для пропущенных занятий это слишком большое значение.
Заменим на среднее значение без учета этих выбросов, дробную часть отбросим.  Пропуски заменим тем же средним.

In [ ]:
df[df.absences>100].absences

In [ ]:
mean_ab = int(df.absences[df.absences<100].mean())
index = df.absences[df.absences>100].index
df.loc[index, 'absences'] = mean_ab
df.absences.fillna(mean_ab, inplace=True)

### Fedu, Medu

In [ ]:
df.groupby('Fedu').Medu.value_counts()

Наблюдается взаимосвязь между переменными Medu и Fedu, для каждого значения Fedu самое частое значение Medu совпадают или находится на втором месте. Поэтому для заполнения пропусков (24) в Fedu присвоим значение Medu из этой же строчки. Наоборот поступим также, там аналогичная зависимость без исключений.

In [ ]:
df.Fedu = df.Fedu.fillna(df.Medu)
df.Medu = df.Medu.fillna(df.Fedu)

### Traveltime

Выбросов нет, 28 пропусков.


In [ ]:
df.groupby(['school', 'address']).traveltime.value_counts(dropna=False)

Школьники из школы GP независимо от типа адреса чаще всего добираются за время < 15 минут, а из MS за 15-30 минут. Заполним пропуски по этому принципу.

In [ ]:
df['traveltime'] = df.apply(lambda x: (1.0 if x.school == 'GP' else 2.0) if pd.isna(x.traveltime) else x.traveltime , axis=1)

### Studytime
Выбросов нет, 7 пропусков. 

In [ ]:
df.groupby('freetime').studytime.value_counts(dropna=False)

Если у ученика много свободного времени после школы, значит он мало занимается после школы. Данные это предположение не отвергают, поэтому заполним **studytime** 1.0, если **freetime** равно 5.0, и значением 2.0 в остальных случаях.

In [ ]:
df['studytime'] = df.apply(lambda x: (1.0 if x.freetime == 5.0 else 2.0) if pd.isna(x.studytime) else x.studytime , axis=1)

### Failures

Выбросов нет. Пропусков - 22. Здесь нулевое значение побеждает с большим отрывом, поэтому заполним пропуск им

In [ ]:
df.failures.fillna(0, inplace=True)

### Famrel
Один выброс - отрицательное значение "-1", просто поменяем ему знак. Пропусков - 27, заполним самым популярным значением этого поля 4.0

In [ ]:
df.loc[df.famrel == -1, 'famrel'] = 1
df.famrel.fillna(4.0, inplace=True)

### Freetime

Выбросов нет. Пропусков - 11. 3.0 - самое частое значение, середина нашей шкалы по этой колонке , поэтому заполним пропуск "золотой" серединой.

In [ ]:
df.freetime.fillna(3.0, inplace=True)

### Goout

Выбросов нет. Пропусков - 8. Остальное все так же как и в **Freetime**


In [ ]:
df.goout.fillna(3.0, inplace=True)

### Health
Выбросов нет. Пропусков - 15, заполним самым популярным значением этого поля 5.0

In [ ]:
df.health.fillna(5.0, inplace=True)

In [ ]:
#=====================================================#
df.info()